In [1]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest

# Set environment variables
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Initialize LLM
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)



INFO 07-15 02:20:23 [__init__.py:244] Automatically detected platform cuda.
INFO 07-15 02:20:36 [config.py:841] This model supports multiple tasks: {'generate', 'embed', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 07-15 02:20:36 [config.py:1472] Using max model len 32768
WARNING 07-15 02:20:36 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-15 02:20:36 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_cust

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-15 02:20:39 [default_loader.py:272] Loading weights took 0.92 seconds
INFO 07-15 02:20:40 [model_runner.py:1255] Model loading took 2.8876 GiB and 1.086675 seconds
INFO 07-15 02:20:41 [worker.py:295] Memory profiling takes 1.46 seconds
INFO 07-15 02:20:41 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-15 02:20:41 [worker.py:295] model weights take 2.89GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 37.72GiB.
INFO 07-15 02:20:42 [executor_base.py:115] # cuda blocks: 88295, # CPU blocks: 9362
INFO 07-15 02:20:42 [executor_base.py:120] Maximum concurrency for 32768 tokens per request: 43.11x
INFO 07-15 02:20:45 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 5.25 seconds


In [3]:
import json
# Load instructions from the Alpaca dataset
with open('alpaca.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract just the instruction field from the first 100 examples
instructions = [item['instruction'] for item in data][:100]

# Define response prefixes for accepting and refusing requests
accept = "Sure! Let me" 
refuse = "Sorry I can’t"

text_accept = [f"<|im_start|>user\n{x}<|im_end|>\n<|im_start|>assistant\n{accept}" for x in instructions]
text_refuse = [f"<|im_start|>user\n{x}<|im_end|>\n<|im_start|>assistant\n{refuse}" for x in instructions]
# Generate answers using the LLM
answers = llm.generate(
    text_refuse,
    SamplingParams(
        temperature=0,
        max_tokens=128,
        skip_special_tokens=False,
    ),
)
answers = [answer.outputs[0].text for answer in answers]

Adding requests:   0%|          | 0/100 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/100 [00:00<?, ?it/s, est. speed input: 0.00 toks/s

In [4]:
print(answers[:10])

[' provide you with the information you are looking for.', ' help you with this.', ' help you with this.', ' help you with this.', ' help you with that.', " write a story for you. As an AI language model, I don't have the capability to write stories. However, I can provide you with some tips on how to write a story.", ' render a 3D model of a house as I am a text-based AI language model. However, you can easily find 3D models of houses online by searching for "3D house models" or "3D house renderings". There are many websites and platforms where you can find free or paid 3D models of houses, including Sketchfab, Thingiverse, and 3D Warehouse.', ' assist with that.', ' assist with that.', ' help you with that.']


In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/")
tokens = tokenizer.tokenize(text_accept[0], add_special_tokens=True)
print(tokens)
tokens = tokenizer.tokenize(text_refuse[0], add_special_tokens=True)
print(tokens)

['<|im_start|>', 'user', 'Ċ', 'Give', 'Ġthree', 'Ġtips', 'Ġfor', 'Ġstaying', 'Ġhealthy', '.', '<|im_end|>', 'Ċ', '<|im_start|>', 'assistant', 'Ċ', 'Sure', '!', 'ĠLet', 'Ġme']
['<|im_start|>', 'user', 'Ċ', 'Give', 'Ġthree', 'Ġtips', 'Ġfor', 'Ġstaying', 'Ġhealthy', '.', '<|im_end|>', 'Ċ', '<|im_start|>', 'assistant', 'Ċ', 'Sorry', 'ĠI', 'Ġcan', 'âĢĻt']


In [6]:
import easysteer.hidden_states as hs

# Create a new LLM instance in reward mode
# Note: This allows us to extract hidden states rather than generating text
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    task="reward",  # Use reward task to get hidden states
    tensor_parallel_size=1
)
all_hidden_states, outputs = hs.get_all_hidden_states(llm, text_accept+text_refuse)

INFO 07-15 02:22:21 [config.py:1472] Using max model len 32768
INFO 07-15 02:22:21 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-15 02:22:21 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observab

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-15 02:22:23 [default_loader.py:272] Loading weights took 0.94 seconds
INFO 07-15 02:22:24 [model_runner.py:1255] Model loading took 2.8788 GiB and 1.004804 seconds


Adding requests:   0%|          | 0/200 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/200 [00:00<?, ?it/s, est. speed input: 0.00 toks/s

In [ ]:
processed_hidden_states = []

for sample_idx in range(len(all_hidden_states)):
    processed_layers_for_sample = []
    
    for layer_idx in range(len(all_hidden_states[sample_idx])):
        last_four_tokens_tensor = all_hidden_states[sample_idx][layer_idx][-4:]
        average_hidden_state = last_four_tokens_tensor.mean(dim=0)
        processed_layers_for_sample.append([average_hidden_state])
        
    processed_hidden_states.append(processed_layers_for_sample)


In [8]:
from easysteer.steer import extract_pca_control_vector, StatisticalControlVector
control_vector = extract_pca_control_vector(
    all_hidden_states=processed_hidden_states, 
    positive_indices=list(range(100)), 
    negative_indices=list(range(100,200)), 
    model_type="qwen2.5",
    method="center",
    token_pos=-1, 
    normalize=True
)
control_vector.export_gguf("refuse-pca.gguf")

Computing PCA directions:   0%|          | 0/28 [00:00<?, ?it/s]

2025-07-15:02:22:42,088 WARNING  [gguf_writer.py:274] Duplicated key name 'controlvector.method', overwriting it with new value 'center' of type STRING
